In [5]:
import pandas as pd
import numpy as np
import math
from sklearn.ensemble import RandomForestRegressor

import cufflinks as cf
%matplotlib inline
cf.go_offline()

pd.set_option('max_columns' , 999)
pd.set_option('max_rows' , 999)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


In [6]:
#리스트 더미화
def making_list_dummy(df, cols) :
    new_df = df.copy()
    for col in cols :
        
        dic = pd.Series(new_df[col].values,index=new_df.index).to_dict()
        s = pd.Series(dic)
        temp = pd.get_dummies(s.apply(pd.Series).stack(), prefix=col).sum(level=0)
        
        new_df = pd.concat([new_df, temp], axis=1)
        
    new_df = new_df.drop(cols, axis=1)
    
    return new_df

In [7]:
#명목형 변수 더미화
def dummify_category_cols(df,cols):
    dummies = []
    new_df = df.copy()

    for idx in range(len(cols)):
        new_df = new_df.join(pd.get_dummies(new_df[cols[idx]], prefix = cols[idx]))
        new_df = new_df.drop(cols[idx], axis =1)
    return new_df

In [8]:
#RMS 구하기
def RMSE(y_true,y_pred):
    
    squaredErrors = (y_pred - y_true)**2
    MSE = squaredErrors.mean()
    
    return math.sqrt(MSE)

In [9]:
df = pd.read_csv("movie_eda.csv", encoding = "cp949")

In [12]:
df.head(2)

,영화명,D+13 휴일수,D+7 휴일수,D-1 시사회수,D-5 시사회수,day,month,naver_not_wanna_see,naver_ratio,naver_wanna_see,naver_리뷰수,runtime,weekday,year,감독,국산,대표국적,등급,미국산,배급사,배우,수입사,영화구분,장르,제작사,D-1 누적매출액,D-1 누적관객수,D-5 누적매출액,D-5 누적관객수,D+1 누적매출액,D+1 누적관객수,D+7 누적관객수,D+13 누적관객수,D-1 상영횟수 합,D-1 스크린수 합,D-1 좌석수 합,D-5 상영횟수 합,D-5 스크린수 합,D-5 좌석수 합,D+1 상영횟수 합,D+1 순위 합,D+1 스크린수 합,D+1 좌석수 합,D-5 상영횟수 평균,D-1 상영횟수 평균,D+1 상영횟수 평균,D-5 순위 평균,D-1 순위 평균,D+1 순위 평균,D-5 스크린수 평균,D-1 스크린수 평균,D+1 스크린수 평균,D-5 좌석수 평균,D-1 좌석수 평균,D+1 좌석수 평균,D-5 관객수 평균,D-1 관객수 평균,D+1 관객수 평균,D-5 매출액 평균,D-1 매출액 평균,D+1 매출액 평균,after2_before28_tweet,after2_before28_like,before4_before34_tweet,before4_before34_like,top500_actors,top500_director,배우점수,감독점수,배급사점수
0,명량,4.0,2.0,9.0,6.0,30,7,1126,0.923739,13639,1676,128.0,2.0,2014,['김한민'],1,한국,15세이상관람가,0,['씨제이이앤엠(주)'],['None'],['None'],1,['사극'],['None'],172231000.0,22500.0,81803000.0,10050.0,1.012452e+10,1410271.0,7329679.0,11775978.0,163.0,145.0,30887.0,87.0,74.0,16924.0,12405.0,2.0,2361.0,2328207.0,14.500000,18.111111,6202.5,15.166667,14.0,1.0,12.333333,16.111111,1180.5,2820.666667,3431.888889,1164103.5,1675.000000,2500.0,705135.5,1.363383e+07,1.913678e+07,5.062260e+09,6593.0,4713.0,5066.0,4464.0,3,1,47.909130,9.114424,98.955292
1,국제시장,5.0,2.0,20.0,17.0,17,12,2052,0.861688,12784,1233,126.0,2.0,2014,['윤제균'],1,한국,12세이상관람가,0,['씨제이이앤엠(주)'],['None'],['None'],1,['드라마'],['None'],256465000.0,33048.0,198916000.0,26194.0,3.069223e+09,416356.0,2312726.0,4869135.0,241.0,227.0,48722.0,191.0,178.0,38223.0,8935.0,3.0,1885.0,1603232.0,11.235294,12.050000,4467.5,26.529412,27.8,1.5,10.470588,11.350000,942.5,2248.411765,2436.100000,801616.0,1540.823529,1652.4,208178.0,1.170094e+07,1.282325e+07,1.534611e+09,2896.0,691.0,1946.0,481.0,4,1,77.181236,8.307558,98.955292


In [16]:
after_col_lst = ['영화명', 'D+13 휴일수', 'D-1 시사회수', 'day', 'month',
       'naver_not_wanna_see', 'naver_ratio', 'naver_wanna_see', 'naver_리뷰수',
       'runtime', 'weekday', 'year', '국산', '대표국적', '등급', '미국산',
       '영화구분', '장르', 'D-1 누적매출액', 'D-1 누적관객수',
       'D+1 누적매출액', 'D+1 누적관객수', 'D+13 누적관객수',
       'D-1 상영횟수 합', 'D-1 스크린수 합', 'D-1 좌석수 합',
       'D+1 상영횟수 합', 'D+1 순위 합', 'D+1 스크린수 합', 'D+1 좌석수 합',
       'D-1 상영횟수 평균', 'D+1 상영횟수 평균', 'D-1 순위 평균',
       'D+1 순위 평균', 'D-1 스크린수 평균', 'D+1 스크린수 평균', "D+1 naver_news",
       'D-1 좌석수 평균', 'D+1 좌석수 평균', 'D-1 관객수 평균', 'D+1 관객수 평균',
       'D-1 매출액 평균', 'D+1 매출액 평균', 'top500_actors', 'top500_director', '배우점수',
       '감독점수', '배급사점수']

before_col_lst = ['영화명', 'D+7 휴일수', 'D-5 시사회수', 'day', 'month',
       'naver_not_wanna_see', 'naver_ratio', 'naver_wanna_see', 'naver_리뷰수',
       'runtime', 'weekday', 'year', '국산', '대표국적', '등급', '미국산',
       '영화구분', '장르', 'D-5 누적매출액',
       'D-5 누적관객수', 'D+7 누적관객수',
       'D-5 상영횟수 합', 'D-5 스크린수 합',
       'D-5 좌석수 합', 
       'D-5 상영횟수 평균', 'D-5 순위 평균', 
       'D-5 스크린수 평균', 'D-5 좌석수 평균',
       'D-5 관객수 평균', "D-5 naver_news",
       'D-5 매출액 평균', 'top500_actors', 'top500_director', '배우점수',
       '감독점수', '배급사점수']

after_df = df[after_col_lst].drop(["대표국적"], axis=1)
before_df = df[before_col_lst].drop(["대표국적"], axis=1)

 # RandomForest 

In [17]:
#개봉 후 영화 예측
new_df = after_df.set_index("영화명", drop=True)
target = "D+13 누적관객수"

In [18]:
#개봉 전 영화 예측
new_df = before_df.set_index("영화명", drop=True)
target = "D+7 누적관객수"

In [19]:
#더미화
new_df = making_list_dummy(new_df, ["장르"])
new_df = dummify_category_cols(new_df, ["등급"])


In [21]:
new_df['is_train'] = np.random.uniform(0, 1, len(new_df)) <= .75

training_set = new_df[new_df['is_train']==True]
test_set = new_df[new_df['is_train']==False]

feature = new_df.columns.drop(target)
target = target

In [22]:
forest = RandomForestRegressor(n_estimators=4500, n_jobs=2)

In [23]:
forest.fit(training_set[feature], training_set[target])

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_split=1e-07, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=4500, n_jobs=2, oob_score=False, random_state=None,
           verbose=0, warm_start=False)

In [24]:
forest.predict(test_set[feature])
forest.score(test_set[feature], test_set[target])

0.77515264211731494

In [34]:
y_pred=forest.predict(test_set[feature])
y_true = test_set[target]
rmse = RMSE(y_true,y_pred)
print(rmse)

719635.579104844


In [35]:
importance = pd.DataFrame(forest.feature_importances_, index = training_set[feature].columns)
(
    importance.assign(header = lambda x: x.index.str.split("_").str[0])
                  .groupby("header")[0]
                  .sum()
                  .iplot(kind="bar")
)

 ### 전처리

In [36]:
#상업영화 이외 버리기
after_df = after_df.drop(after_df[after_df["영화구분"] != 1].index)
before_df = before_df.drop(before_df[before_df["영화구분"] != 1].index)

after_df = after_df.drop(["영화구분"], axis=1)
before_df = before_df.drop(["영화구분"], axis=1)

In [37]:
#미국/한국 영화 이외 버리기
after_df = after_df.drop(after_df[after_df["국산"]+after_df["미국산"] == 0].index)
before_df = before_df.drop(before_df[before_df["국산"]+before_df["미국산"] == 0].index)

### 장르별

In [57]:
#개봉후 영화 예측
after_df2 = after_df.set_index("영화명", drop=True)

after_df2["genre"] = after_df2["장르"].map(lambda x: 1 if "액션" in x else 0)
after_df2 = after_df2.drop(after_df2[after_df2["genre"]==1].index)
after_df2 = after_df2.drop(["genre"], axis=1)

after_df2 = making_list_dummy(after_df2, ["장르"])
new_df = dummify_category_cols(after_df2,["등급"])

target = "D+13 누적관객수"

In [75]:
#개봉전 영화 예측
before_df2 = before_df.set_index("영화명", drop=True)

before_df2["genre"] = before_df2["장르"].map(lambda x: 1 if "사극" in x else 0)
before_df2 = before_df2.drop(before_df2[before_df2["genre"]==1].index)
before_df2 = before_df2.drop(["genre"], axis=1)

before_df2 = making_list_dummy(before_df2, ["장르"])
new_df = dummify_category_cols(before_df2,["등급"])

target = "D+7 누적관객수"

In [76]:
new_df['is_train'] = np.random.uniform(0, 1, len(new_df)) <= .75

training_set = new_df[new_df['is_train']==True]
test_set = new_df[new_df['is_train']==False]

feature = new_df.columns.drop(target)
target = target

In [77]:
forest = RandomForestRegressor(n_estimators=4500, n_jobs=2)

In [78]:
forest.fit(training_set[feature], training_set[target])

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_split=1e-07, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=4500, n_jobs=2, oob_score=False, random_state=None,
           verbose=0, warm_start=False)

In [79]:
forest.predict(test_set[feature])
forest.score(test_set[feature], test_set[target])

0.87075242875735759

In [80]:
y_pred=forest.predict(test_set[feature])
y_true = test_set[target]
rmse = RMSE(y_true,y_pred)
print(rmse)

365321.9424045749


### 국가별

In [51]:
#개봉후 영화 예측
after_df2 = after_df.set_index("영화명", drop=True)

after_df2 = after_df2[after_df2["미국산"]==1]
after_df2 = after_df2.drop(["미국산", "국산"], axis=1)

after_df2 = making_list_dummy(after_df2, ["장르"])
new_df = dummify_category_cols(after_df2, ["등급"])

target = "D+13 누적관객수"

In [81]:
#개봉전 영화 예측
before_df2 = before_df.set_index("영화명", drop=True)

before_df2 = before_df2[before_df2["국산"]==1]
before_df2 = before_df2.drop(["미국산", "국산"], axis=1)

before_df2 = making_list_dummy(before_df2, ["장르"])
new_df = dummify_category_cols(before_df2,["등급"])

target = "D+7 누적관객수"

In [82]:
new_df['is_train'] = np.random.uniform(0, 1, len(new_df)) <= .75

training_set = new_df[new_df['is_train']==True]
test_set = new_df[new_df['is_train']==False]

feature = new_df.columns.drop(target)
target = target

In [83]:
forest = RandomForestRegressor(n_estimators=4500, n_jobs=2)

In [84]:
forest.fit(training_set[feature], training_set[target])

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_split=1e-07, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=4500, n_jobs=2, oob_score=False, random_state=None,
           verbose=0, warm_start=False)

In [85]:
forest.predict(test_set[feature])
forest.score(test_set[feature], test_set[target])

0.90355294841899769

In [86]:
y_pred=forest.predict(test_set[feature])
y_true = test_set[target]
rmse = RMSE(y_true,y_pred)
print(rmse)

353100.3401991176
